In [1]:
import torch
import pickle
import os
import numpy as np
from dataset_utils import PickleDataset, get_concepts_numpy, Experiment
from eegatscale.models import LinearHeadBENDR
from eegatscale.transforms import Standardize

from captum.concept import TCAV, Concept

## Test loading of random MMIDB concept

In [2]:
concepts_dir = '/scratch/s194101/concepts/'
left_concept_path = os.path.join(concepts_dir, "random_left_concept_MMIDB")

In [3]:
left_concept_path

'/scratch/s194101/concepts/random_left_concept_MMIDB'

In [4]:
temp = PickleDataset(left_concept_path, remove_last_channel=True)
len(temp)

200

In [5]:
dataloader = torch.utils.data.DataLoader(temp, batch_size=10, shuffle=True)

In [6]:
# test shape
for batch in dataloader:
    print(batch.shape)
    print(batch.mean().item(), batch.std().item(), batch.min().item(), batch.max().item())
    break

torch.Size([10, 19, 1024])
-0.24066728353500366 4.990466594696045 -5.0 5.0


## Test loading og TUH concept

In [7]:
concepts_dir = '/scratch/s194101/concepts/'
tuh_concept_path = os.path.join(concepts_dir, "TUH_clean_alpha_subtract_Truesigma_Trueabs_100.0_142440_131124/Alpha_Somatosensory and Motor Cortex-lh")


In [8]:
tuh_concept_path

'/scratch/s194101/concepts/TUH_clean_alpha_subtract_Truesigma_Trueabs_100.0_142440_131124/Alpha_Somatosensory and Motor Cortex-lh'

In [9]:
temp_tuh = PickleDataset(tuh_concept_path, remove_last_channel=False)
len(temp_tuh)

32

In [10]:
dataloader_tuh = torch.utils.data.DataLoader(temp_tuh, batch_size=10, shuffle=True)

In [11]:
# test shape
for batch in dataloader_tuh:
    print(batch.shape)
    print(batch.mean().item(), batch.std().item(), batch.min().item(), batch.max().item())
    break

torch.Size([10, 19, 1024])
0.003646446835935451 2.3420873176937063 -5.0 5.0


## Test loading of new mmidb concept after i changed channels and normalization in the loop

In [12]:
concepts_dir = '/scratch/s194101/concepts/'
mmidb_concept_path_new = os.path.join(concepts_dir, "sanity_check_concepts_MMIDB_alpha_new/Alpha_Somatosensory and Motor Cortex-lh")


In [13]:
mmidb_concept_path_new

'/scratch/s194101/concepts/sanity_check_concepts_MMIDB_alpha_new/Alpha_Somatosensory and Motor Cortex-lh'

In [14]:
temp_mmidb_new = PickleDataset(mmidb_concept_path_new, remove_last_channel=False)
len(temp_mmidb_new)

FileNotFoundError: [Errno 2] No such file or directory: '/scratch/s194101/concepts/sanity_check_concepts_MMIDB_alpha_new/Alpha_Somatosensory and Motor Cortex-lh'

In [ ]:
dataloader_mmidb_new = torch.utils.data.DataLoader(temp_mmidb_new, batch_size=10, shuffle=True)

In [ ]:
# test shape
for batch in dataloader_mmidb_new:
    print(batch.shape)
    print(batch.mean().item(), batch.std().item(), batch.min().item(), batch.max().item())
    break

torch.Size([10, 19, 1024])
-0.20254355669021606 2.5949857234954834 -5.0 5.0


## load model

In [8]:
out_features = 2
# encoder_path = "/scratch/agjma/checkpoints/tuh-noica-standardize-epoch=9-step=68317-val_loss=0.26.ckpt"
encoder_path = "/scratch/s194101/finetune_logs_all/mmidb_noica_noica_5.0_splits10_repeat2/version_19/checkpoints/epoch=7-step=8120.ckpt"

model = LinearHeadBENDR(encoder_path, encoder_h=512, in_features=19, out_features=out_features)
state_dict = torch.load(encoder_path, map_location=torch.device('cpu'))["state_dict"]
model.load_state_dict(state_dict)
model.eval()
print("model loaded")

/home/s194101/.conda/envs/myenv/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/s194101/Bachelors-explainability/eegatscale/models/linearheadbendr.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We r

Initialized mask embedding and position encoder from  /scratch/s194101/finetune_logs_all/mmidb_noica_noica_5.0_splits10_repeat2/version_19/checkpoints/epoch=7-step=8120.ckpt
model loaded


/home/s194101/Bachelors-explainability/eegatscale/layers.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(filename, map_location=torch.device("cp

In [9]:
exp_1 = Experiment(model, concepts_dir, ["random_left_concept_MMIDB", "random_mmidb"], verbose=True)
exp_2 = Experiment(model, concepts_dir, ["random_left_concept_MMIDB", "random_mmidb"], verbose=True)

Loaded dataset random_left_concept_MMIDB with 200 samples
Loaded dataset random_mmidb with 200 samples
Loaded dataset random_left_concept_MMIDB with 200 samples
Loaded dataset random_mmidb with 200 samples
